In [18]:
import pandas as pd

# Data loading and cleaning

In [19]:
df_nov = pd.read_csv('2019-Nov.csv')
df_oct = pd.read_csv('2019-Oct.csv')
df_feb = pd.read_csv('2020-Feb.csv')
df_jan = pd.read_csv('2020-Jan.csv')

df_nov = df_nov[['user_id', 'event_time']]
df_oct = df_oct[['user_id', 'event_time']]
df_feb = df_feb[['user_id', 'event_time']]
df_jan = df_jan[['user_id', 'event_time']]

df = pd.concat([df_nov, df_oct, df_feb, df_jan])
df.reset_index(inplace = True, drop = True)

df

,user_id,event_time
0,562076640,2019-11-01 00:00:02 UTC
1,553329724,2019-11-01 00:00:09 UTC
2,556138645,2019-11-01 00:00:10 UTC
3,564506666,2019-11-01 00:00:11 UTC
4,553329724,2019-11-01 00:00:24 UTC
...,...,...
17159549,607092857,2020-01-31 23:59:52 UTC
17159550,607092857,2020-01-31 23:59:52 UTC
17159551,423651741,2020-01-31 23:59:53 UTC
17159552,607092857,2020-01-31 23:59:57 UTC


In [20]:
def convert_time(data):
    return int(data[:4] + data[5:7])

df['event_time'] = df['event_time'].map(convert_time)

df

,user_id,event_time
0,562076640,201911
1,553329724,201911
2,556138645,201911
3,564506666,201911
4,553329724,201911
...,...,...
17159549,607092857,202001
17159550,607092857,202001
17159551,423651741,202001
17159552,607092857,202001


# First step - including first order date

In [21]:
df.sort_values(['user_id'], inplace = True, ignore_index = True)
df

,user_id,event_time
0,465496,202001
1,465496,202001
2,465496,202001
3,1120748,201911
4,1458813,201911
...,...,...
17159549,622090043,202002
17159550,622090052,202002
17159551,622090052,202002
17159552,622090098,202002


In [22]:
def date_of_the_first_purcha(df):

    first_date = df.groupby('user_id')['event_time'].min().reset_index()
    first_date = first_date.rename(columns={'event_time': 'first_purchase'})
    df = df.merge(first_date, on='user_id', how='left')

    return df

df = date_of_the_first_purcha(df)
df

,user_id,event_time,first_purchase
0,465496,202001,202001
1,465496,202001,202001
2,465496,202001,202001
3,1120748,201911,201911
4,1458813,201911,201911
...,...,...,...
17159549,622090043,202002,202002
17159550,622090052,202002,202002
17159551,622090052,202002,202002
17159552,622090098,202002,202002


In [29]:
df['event_time'] = df['event_time'].astype(str)
df['first_purchase'] = df['first_purchase'].astype(str)
df

,user_id,event_time,first_purchase
0,465496,202001,202001
1,1120748,201911,201911
2,1458813,201911,201911
3,2038666,202002,202002
4,3571443,202002,202002
...,...,...,...
1569019,622090016,202002,202002
1569020,622090043,202002,202002
1569021,622090052,202002,202002
1569022,622090098,202002,202002


In [30]:
df.drop_duplicates(inplace = True, ignore_index = True)

# Second model - Cohort Analysis

In [31]:
grouping = df.groupby(['event_time', 'first_purchase'])

cohort_data = grouping['user_id'].apply(pd.Series.nunique)
cohort_data

event_time  first_purchase
201910      201910            399664
201911      201910             54796
            201911            313436
202001      201910             31750
            201911             21942
            202001            356381
202002      201910             26953
            201911             17424
            202001             40668
            202002            306010
Name: user_id, dtype: int64

In [32]:
cohort_data = cohort_data.reset_index()
cohort_counts = cohort_data.pivot(index = 'first_purchase',
                                    columns = 'event_time',
                                    values = 'user_id')

In [34]:
cohort_counts

event_time,201910,201911,202001,202002
first_purchase,,,,
201910,399664.0,54796.0,31750.0,26953.0
201911,NaN,313436.0,21942.0,17424.0
202001,NaN,NaN,356381.0,40668.0
202002,NaN,NaN,NaN,306010.0
